In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 90% !important; }</style>"))

In [3]:
# library import 
import os, sys
import jaydebeapi as jdb
import pandas as pd
import numpy as np
import seaborn as sns
import datetime, time
import joblib

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import f1_score, roc_auc_score, recall_score, precision_score

import warnings
warnings.filterwarnings('ignore')

In [7]:
# 가맹점 별 운영정보 데이터
oil_reg_type_anl_ds = pd.read_csv(r".\b_da_oil_reg_type_anl_ds.csv")
oil_reg_type_anl_ds.drop(["레코드 수"], axis= 1, inplace = True)

oil_reg_type_anl_ds.shape

(2911060, 16)

In [8]:
oil_reg_type_anl_ds = oil_reg_type_anl_ds[['reg_no', 'app_amt', 'app_cnt', 'all_day', 'member_no', 'weekday', 'weekend',
'avg_app_amt','avg_oil_amt', 'avg_oil_prc', 'commute', 'gungu_avg_prc',
'max_reg_use_cnt', 'oil_amt', 'oil_prc_diff', 'use_reg_cnt']]

In [9]:
oil_cust_dat = oil_reg_type_anl_ds.drop(["app_amt", "oil_amt", "gungu_avg_prc"], axis = 1)

## base table 
##### 6개월간 가맹점에 6번 이상 반복해서 방문, 평균 8L 이상 주유, 평균 주유량 80L 이상 이상치로 판단 제거 
* 방문 기준: 카드번호

#### 1) 단골/일반 

In [10]:
ym = datetime.datetime.now().strftime("%Y%m")

dangol_cust = oil_cust_dat.query("app_cnt >=6 & avg_oil_amt >= 8 & avg_oil_amt <= 80").loc[:,["member_no", "use_reg_cnt", "app_cnt" ,"max_reg_use_cnt"]]

dangol_cust['max_reg_use_rto'] = dangol_cust['max_reg_use_cnt'] / dangol_cust['app_cnt']
dangol_cust = dangol_cust.drop(["member_no", "app_cnt"], axis = 1 )

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

dangol_cust_md = pd.DataFrame(scaler.fit_transform(dangol_cust), columns = dangol_cust.columns)
kmeans = KMeans(n_clusters=2)

kmeans.fit(dangol_cust_md.values)
y_kmeans = kmeans.predict(dangol_cust_md.values)

dangol_cust_md['cluster'] = y_kmeans
dangol_cust['cluster'] = y_kmeans

md_dangol_cust = dangol_cust.copy()

y = md_dangol_cust['cluster'].values
X = md_dangol_cust.drop('cluster', axis = 1).values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)

# Random Forest Model
rf = RandomForestClassifier(n_estimators= 100)
rf.fit(X_train, y_train)
y_prob = rf.predict_proba(X_test)
y_pred = (y_prob[:,1] > 0.9).astype("int")

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm_matrix = pd.DataFrame(data=cm, columns=['Actual Positive:1', 'Actual Negative:0'], 
                                 index=['Predict Positive:1', 'Predict Negative:0'])
sns.heatmap(cm_matrix, annot=True, fmt='d', cmap='YlGnBu')

In [ ]:
joblib.dump(rf, f'regular_rf_{ym}.pkl')

#### 2) 출퇴근/종일 

In [ ]:
com_all_dat = oil_cust_dat.query("app_cnt >=6 & avg_oil_amt >= 8 & avg_oil_amt <= 80").loc[:,["commute", "all_day"]]
com_all_dat['commute_rto'] = com_all_dat['commute'] / com_all_dat['all_day']

com_all_dat_md = pd.DataFrame(scaler.fit_transform(com_all_dat), columns = com_all_dat.columns)
km_com_all = KMeans(n_clusters=2)

km_com_all.fit(com_all_dat_md.values)
y_kmeans = km_com_all.predict(com_all_dat_md.values)

com_all_dat_md['cluster'] = y_kmeans
com_all_dat['cluster'] = y_kmeans

y = com_all_dat['cluster'].values
X = com_all_dat.drop('cluster', axis = 1).values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)

# Random Forest Model
rf = RandomForestClassifier(n_estimators= 100)
rf.fit(X_train, y_train)
y_prob = rf.predict_proba(X_test)
y_pred = (y_prob[:,1] > 0.9).astype("int")

cm = confusion_matrix(y_test, y_pred)
cm_matrix = pd.DataFrame(data=cm, columns=['Actual Positive:1', 'Actual Negative:0'], 
                                 index=['Predict Positive:1', 'Predict Negative:0'])
sns.heatmap(cm_matrix, annot=True, fmt='d', cmap='YlGnBu')

joblib.dump(rf, f'commute_all_rf_{ym}.pkl')

#### 3) 고액(가득)/알뜰형(정액, 정량)

In [ ]:
full_min_dat = oil_cust_dat.query("app_cnt >=6 & avg_oil_amt >= 8 & avg_oil_amt <= 80").loc[:,["avg_app_amt", "avg_oil_amt", "oil_prc_diff", "avg_oil_prc"]]

full_min_dat_md = pd.DataFrame(scaler.fit_transform(full_min_dat), columns = full_min_dat.columns)
km_full_min = KMeans(n_clusters=2)

km_full_min.fit(full_min_dat_md.values)
y_kmeans = km_full_min.predict(full_min_dat_md.values)

full_min_dat_md['cluster'] = y_kmeans
full_min_dat['cluster'] = y_kmeans

y = full_min_dat['cluster'].values
X = full_min_dat.drop('cluster', axis = 1).values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)

# Random Forest Model
rf = RandomForestClassifier(n_estimators= 100)
rf.fit(X_train, y_train)
y_prob = rf.predict_proba(X_test)
y_pred = (y_prob[:,1] > 0.9).astype("int")

cm = confusion_matrix(y_test, y_pred)
cm_matrix = pd.DataFrame(data=cm, columns=['Actual Positive:1', 'Actual Negative:0'], 
                                 index=['Predict Positive:1', 'Predict Negative:0'])
sns.heatmap(cm_matrix, annot=True, fmt='d', cmap='YlGnBu')

joblib.dump(rf, f'full_min_rf_{ym}.pkl')

#### 4) 주중/ 주말형

In [ ]:
holy_dat = oil_cust_dat.query("app_cnt >=6 & avg_oil_amt >= 8 & avg_oil_amt <= 80").loc[:,["weekday", "weekend"]]
holy_dat['holy_rto'] = holy_dat['weekend'] / (holy_dat['weekend'] + holy_dat['weekday'])

holy_dat_md = pd.DataFrame(scaler.fit_transform(holy_dat), columns = holy_dat.columns)
km_holy = KMeans(n_clusters=2)

km_holy.fit(holy_dat_md.values)
y_kmeans = km_holy.predict(holy_dat_md.values)

holy_dat_md['cluster'] = y_kmeans
holy_dat['cluster'] = y_kmeans

y = holy_dat['cluster'].values
X = holy_dat.drop('cluster', axis = 1).values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)

# Random Forest Model
rf = RandomForestClassifier(n_estimators= 100)
rf.fit(X_train, y_train)
y_prob = rf.predict_proba(X_test)
y_pred = (y_prob[:,1] > 0.9).astype("int")

cm = confusion_matrix(y_test, y_pred)
cm_matrix = pd.DataFrame(data=cm, columns=['Actual Positive:1', 'Actual Negative:0'], 
                                 index=['Predict Positive:1', 'Predict Negative:0'])
sns.heatmap(cm_matrix, annot=True, fmt='d', cmap='YlGnBu')

joblib.dump(rf, f'holy_rf_{ym}.pkl')